In [18]:
import streamlit as st
from datetime import datetime, timedelta
from yahooquery import Ticker
from pathlib import Path
import pandas as pd
from os.path import exists
import time
from src.data import yahoo_fin_stock_info as si
import pickle5 as pickle

from src.models.recommender.simple_rec import Simple_Rick
import src.models.portfolio.proof as p1
import src.models.portfolio.proof_port as p2
from src.models.portfolio.web_pca import The_PCA_Analysis as pca
from src.models.portfolio.proof_port import The_Portfolio_Optimizer
from src.models.portfolio.proof import Proof_of_Concept
from src.models.portfolio.web_monteCarloCholesky import MonteCarloCholesky as mcc
from src.gmail import The_Only_Mailer

# pd.options.display.max_rows = 999

In [19]:
july_2021 = [
#     'MONDAY',   'TUESDAY',   'WEDNESDAY',  'THURSDAY',     'FRIDAY'
#  |------------|-------------|-------------|-------------|--------------|
                                '2021-07-14', '2021-07-15', '2021-07-16',
    '2021-07-19', '2021-07-20', '2021-07-21', '2021-07-22', '2021-07-23',
    '2021-07-26', '2021-07-27',               
    '2021-07-29', '2021-07-30',
]

august_2021 = [
#     'MONDAY',   'TUESDAY',   'WEDNESDAY',  'THURSDAY',     'FRIDAY'
#  |------------|-------------|--------------|-------------|--------------|
    '2021-08-02', '2021-08-03', '2021-08-04', '2021-08-05', '2021-08-06',
    '2021-08-09', '2021-08-10',                             '2021-08-13',
    '2021-08-16',                                           '2021-08-20',
    '2021-08-23',                             '2021-08-26', '2021-08-27',
                  '2021-08-31',
]

september_2021 = [
#     'MONDAY',   'TUESDAY',   'WEDNESDAY',  'THURSDAY',     'FRIDAY'
#  |------------|-------------|-------------|-------------|--------------|
                                              '2021-09-02', '2021-09-03',
                                '2021-09-08', '2021-09-09', '2021-09-10',
    '2021-09-13', '2021-09-14', '2021-09-15', '2021-09-16', '2021-09-17',
                  '2021-09-21', '2021-09-22', '2021-09-23', '2021-09-24',
    '2021-09-27', '2021-09-28',
]

october_2021 = [
#     'MONDAY',   'TUESDAY',   'WEDNESDAY',  'THURSDAY',     'FRIDAY'
#  |------------|-------------|-------------|-------------|--------------|
                                                            '2021-10-01',
    '2021-10-04', '2021-10-05', '2021-10-06', '2021-10-07', '2021-10-08',
    '2021-10-11',               '2021-10-13', '2021-10-14', '2021-10-15',
    '2021-10-18', '2021-10-19', '2021-10-20', '2021-10-21', '2021-10-22',
    '2021-10-25',               '2021-10-27', '2021-10-28', 
                                                            '2021-10-29',
]

november_2021 = [
#     'MONDAY',   'TUESDAY',   'WEDNESDAY',  'THURSDAY',     'FRIDAY'
#  |------------|-------------|-------------|-------------|--------------|
    '2021-11-01', '2021-11-02', '2021-11-03',               '2021-11-05',
                                '2021-11-10', '2021-11-11', '2021-11-12',
    '2021-11-15', '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-19',
                  '2021-11-23', '2021-11-24', '2021-11-25', '2021-11-26',
]

december_2021 = [
#     'MONDAY',   'TUESDAY',   'WEDNESDAY',  'THURSDAY',     'FRIDAY'
#  |------------|-------------|-------------|-------------|--------------|
                                '2021-12-01', '2021-12-02', '2021-12-03',
    '2021-12-06', '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10',
    '2021-12-13', '2021-12-14', '2021-12-15', '2021-12-16', '2021-12-17',
    '2021-12-20', '2021-12-21',                             '2021-12-24',
]

january_2022 = [
#     'MONDAY',   'TUESDAY',   'WEDNESDAY',  'THURSDAY',     'FRIDAY'
#  |------------|-------------|-------------|-------------|--------------|
    '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07',
    '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14',
                  '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21',
                                '2022-01-26', '2022-01-27', '2022-01-28',
]

february_2022 = [
#     'MONDAY',   'TUESDAY',   'WEDNESDAY',  'THURSDAY',     'FRIDAY'
#  |------------|-------------|-------------|-------------|--------------|
                                                            '2022-02-04',
    '2022-02-07',               '2022-02-09', '2022-02-10', '2022-02-11',
    '2022-02-14', '2022-02-15', '2022-02-16',               '2022-02-18',
                  '2022-02-22', '2022-02-23', '2022-02-24', '2022-02-25',
]

march_2022 = [
#     'MONDAY',   'TUESDAY',   'WEDNESDAY',  'THURSDAY',     'FRIDAY'
#  |------------|-------------|-------------|-------------|--------------|
                 '2022-03-01',                              '2022-03-04',
    '2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10', '2022-03-11',    
]


all_dates_list = [
    july_2021,
    august_2021,
    september_2021,
    october_2021, 
    november_2021,
    december_2021,
    january_2022,
    february_2022,
    march_2022,
]

days = []
for date_lists in all_dates_list:
    days += date_lists

In [20]:
def single_day(day):
    month = str(day)[:7]
    year = str(day)[:4]
    saveRec = Path(f"/home/gdp/russell_3k/data/recommenders/{year}/{month}/{day}/recommender_04_return_dataFrame.pkl")

    df = pd.read_pickle(saveRec).round(2)
    print(len(df))

    sp500_ticker_lst = si.tickers_sp500()
    df['in_sp500'] = df['ticker'].isin(sp500_ticker_lst)
    df1 = df[df['in_sp500'] == True]

    print(len(df1))
    return df1
    
# fd = single_day('2022-02-25')
# fd.head()

In [21]:
def all_days():
    for day in days:
        try:
            month = str(day)[:7]
            year = str(day)[:4]
            saveRec = Path(f"/home/gdp/russell_3k/data/recommenders/{year}/{month}/{day}/recommender_04_return_dataFrame.pkl")
            russell_3k = Path(f"data/russell_3k/{year}/{month}/{day}/")
            if not russell_3k.exists():
                russell_3k.mkdir(parents=True)    

            with open(saveRec, "rb" ) as fh:
                data = pd.DataFrame(pickle.load(fh)).round(2)
            
            data.columns = [x.lower() for x in data.columns]
            data = data.rename(columns={'symbol': 'ticker'})
            
            russell_1k_stocks = si.tickers_russell1000()
            russell_2k_stocks = list(pd.read_csv("data/ticker_lists/russell_2k_stock_lst.csv")['Ticker'])
            russell_3k_stocks = russell_1k_stocks + russell_2k_stocks            
            
            data['in_russell_3k'] = data['ticker'].isin(russell_3k_stocks)
            df = data[data['in_russell_3k'] == True]
            df.to_csv(russell_3k / f"russell_3k_recommender.csv")
        
        except Exception:
            print(day, " - russell_3k")
            pass   

In [22]:
all_days()

2022-03-08  - russell_3k
2022-03-09  - russell_3k
2022-03-10  - russell_3k
2022-03-11  - russell_3k


In [23]:
# check = pd.read_csv("/home/gdp/russell_3k/data/russell_3k/2021/2021-07/2021-07-14/sp500_recommender.csv")
# print(len(check))
# check

In [28]:
# x = pd.read_pickle("data/russell_3k/2021/2021-07/2021-07-15/df_train_data.pkl")
# pd.to_datetime(x.index[-1])

In [ ]:
sp500_ticker_lst = si.tickers_sp500()

In [ ]:
def check_stock(ticker):
    if ticker in sp500_ticker_lst:
        print('YES')
    else:
        print('Fuck NO!')

In [ ]:
stock = "BNED"
check_stock(stock)

Fuck NO!


In [ ]:
len(si.tickers_dow())

30

In [ ]:
len(si.tickers_sp100())

101

In [ ]:
len(si.tickers_sp400())

400

In [ ]:
len(si.tickers_sp500())

505

In [ ]:
len(si.tickers_sp600())

601

In [ ]:
len(si.tickers_sp600()) + len(si.tickers_sp400())

1001

In [ ]:
len(si.tickers_russell1000())

1025

In [ ]:
# len(si.tickers_russell3000())

3051

In [ ]:
len(si.tickers_nasdaq())

5598

In [ ]:
import pandas_datareader as pdr

nasdaq_tickers_df = pdr.nasdaq_trader.get_nasdaq_symbols(retry_count=3, timeout=30, pause=None)
nasdaq_tickers_df.reset_index(inplace=True)

new_df = nasdaq_tickers_df.drop_duplicates(subset='Symbol', keep="first")

new_df = new_df[new_df['ETF'] != True]
new_df = new_df[new_df['Test Issue'] != True]

new_df["Symbol"] = new_df["Symbol"].str.replace(".", "-")
new_df["Symbol"] = new_df["Symbol"].str.replace("-U", "")
new_df["Symbol"] = new_df["Symbol"].str.replace("-W", "")
new_df["Symbol"] = new_df["Symbol"].str.replace("-A", "")
new_df["Symbol"] = new_df["Symbol"].str.replace("-B", "")
new_df["Symbol"] = new_df["Symbol"].str.replace("-C", "")
new_df["Symbol"] = new_df["Symbol"].str.replace("-R", "")
new_df["Symbol"] = new_df["Symbol"].str.replace("-V", "")

new_df = new_df.drop_duplicates(subset='Symbol', keep="first")

new_df

,Symbol,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
0,A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.0,False,NaN,A,A,False
1,AA,True,Alcoa Corporation Common Stock,N,,False,100.0,False,NaN,AA,AA,False
4,AAC,True,Ares Acquisition Corporation Class A Ordinary Shares,N,,False,100.0,False,NaN,AAC,AAC,False
7,AACG,True,"ATA Creativity Global - American Depositary Shares, each representing two common shares",Q,G,False,100.0,False,N,NaN,AACG,False
8,AACI,True,Armada Acquisition Corp. I - Common Stock,Q,G,False,100.0,False,N,NaN,AACI,False
...,...,...,...,...,...,...,...,...,...,...,...,...
12045,ZWS,True,Zurn Water Solutions Corporation Common Stock,N,,False,100.0,False,NaN,ZWS,ZWS,False
12050,ZY,True,Zymergen Inc. - Common Stock,Q,Q,False,100.0,False,N,NaN,ZY,False
12051,ZYME,True,Zymeworks Inc. Common Shares,N,,False,100.0,False,NaN,ZYME,ZYME,False
12052,ZYNE,True,"Zynerba Pharmaceuticals, Inc. - Common Stock",Q,G,False,100.0,False,N,NaN,ZYNE,False


In [ ]:
from stocksymbol import StockSymbol

api_key = '88d20a9f-1e57-4434-b162-1a73ccf399f3'
ss = StockSymbol(api_key)

In [ ]:
print(len(ss.get_symbol_list(index="IXIC", symbols_only=True)))
# pd.DataFrame.from_dict(ss.get_symbol_list(index="IXIC"))

1500


In [ ]:
len(pd.DataFrame.from_dict(ss.get_symbol_list(index="MID")))

399

In [ ]:
len(pd.DataFrame.from_dict(ss.get_symbol_list(index="XMI")))

20

In [ ]:
y = pd.DataFrame.from_dict(ss.get_index_list())
y=y[y['market']=='america']
y = y.sort_values('indexName')
y.tail(25)

,market,abbreviation,indexId,indexName
88,america,us,INDS,NASDAQ INDUSTRIAL
67,america,us,INSR,NASDAQ INSURANCE
101,america,us,OFIN,NASDAQ OTHER FINANCE
95,america,us,IXTC,NASDAQ TELECOMMUNICATIONS
94,america,us,TRAN,NASDAQ TRANSPORTATION
82,america,us,XMI,NYSE ARCA MAJOR MARKET
107,america,us,XAU,PHLX GOLD AND SILVER SECTOR INDEX
99,america,us,HGX,PHLX HOUSING SECTOR
97,america,us,OSX,PHLX OIL SERVICE SECTOR
78,america,us,SOX,PHLX SEMICONDUCTOR


In [ ]:
pd.DataFrame.from_dict(ss.get_symbol_list(index="XMI"))

,symbol,shortName,longName,exchange,market,quoteType
0,MSFT,microsoft,Microsoft Corporation,NASDAQ,us_market,EQUITY
1,JPM,jpmorgan-chase,JP Morgan Chase & Co.,NYSE,us_market,EQUITY
2,JNJ,johnson-and-johnson,Johnson & Johnson,NYSE,us_market,EQUITY
3,WMT,walmart,Walmart Inc.,NYSE,us_market,EQUITY
4,PG,procter-and-gamble,Procter & Gamble Company (The),NYSE,us_market,EQUITY
5,XOM,exxon,Exxon Mobil Corporation,NYSE,us_market,EQUITY
6,DIS,walt-disney,Walt Disney Company (The),NYSE,us_market,EQUITY
7,KO,coca-cola,Coca-Cola Company (The),NYSE,us_market,EQUITY
8,CVX,chevron,Chevron Corporation,NYSE,us_market,EQUITY
9,WFC,wells-fargo,Wells Fargo & Company,NYSE,us_market,EQUITY


In [ ]:
len(si.tickers_other())

5431

In [ ]:
import pandas as pd
from yahoo_fin import stock_info as si

In [ ]:
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

In [ ]:
sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

In [ ]:
symbols = set.union( sym1, sym2, sym3, sym4 )

In [ ]:
my_list = ['W', 'R', 'P', 'Q']

In [ ]:
del_set = set()
sav_set = set()

In [ ]:
for symbol in symbols:
    if len( symbol ) > 4 and symbol[-1] in my_list:
        del_set.add( symbol )
    else:
        sav_set.add( symbol )

In [ ]:
print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

Removed 1097 unqualified stock symbols...
There are 10960 qualified stock symbols...


In [ ]:
import pandas as pd
from yahoo_fin import stock_info as si


# gather stock symbols from major US exchanges
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

# convert DataFrame to list, then to sets
sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

# join the 4 sets into one. Because it's a set, there will be no duplicate symbols
symbols = set.union( sym1, sym2, sym3, sym4 )

# Some stocks are 5 characters. Those stocks with the suffixes listed below are not of interest.
my_list = ['W', 'R', 'P', 'Q', 'A', 'B', 'H', 'E', 'C', 'J']
del_set = set()
sav_set = set()

for symbol in symbols:
    if len( symbol ) > 4 and symbol[-1] in my_list:
        del_set.add( symbol )
    else:
        sav_set.add( symbol )

print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

Removed 1413 unqualified stock symbols...
There are 10644 qualified stock symbols...


In [ ]:
symbols = [x.replace('$', '-') for x in symbols]

In [ ]:
fix = []
for s in symbols:
    lst = list(s)
    for l in lst:
        if l == '-':
            fix.append((s))

In [ ]:
len(fix)

455

In [ ]:
lst1=[]
for f in fix:
    if len(f) <= 6:
        lst1.append(f[-1])
    else:
        print(f)

In [ ]:
sorted(lst1)

['-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C'

In [ ]:

# !pip install yfinance # install yfinance
# importing needed libraries/modules
import numpy as np 
import pandas as pd 
import yfinance as yf

In [ ]:
url = "https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list"
# url = "https://finance.yahoo.com/world-indices/"

In [ ]:
# Retrieving List of World Major Stock Indices from Yahoo! Finance
df_list = pd.read_html(url)
majorStockIdx = df_list[0]
majorStockIdx.head()

,Bank NameBank,CityCity,StateSt,CertCert,Acquiring InstitutionAI,Closing DateClosing,FundFund
0,Almena State Bank,Almena,KS,15426,Equity Bank,"October 23, 2020",10538
1,First City Bank of Florida,Fort Walton Beach,FL,16748,"United Fidelity Bank, fsb","October 16, 2020",10537
2,The First State Bank,Barboursville,WV,14361,"MVB Bank, Inc.","April 3, 2020",10536
3,Ericson State Bank,Ericson,NE,18265,Farmers and Merchants Bank,"February 14, 2020",10535
4,City National Bank of New Jersey,Newark,NJ,21111,Industrial Bank,"November 1, 2019",10534
